# Utah HB 15 - Example Household Impacts

This notebook shows how Utah's proposed Medicaid expansion repeal (HB 15) would affect different household types.

**Key takeaway:** Whether someone falls into the "coverage gap" or transitions to ACA depends primarily on their income relative to the Federal Poverty Level (FPL):
- Below 100% FPL → Coverage gap (no ACA available)
- 100-138% FPL → Can transition to ACA subsidies

**Note:** These examples assume federal Medicaid work requirements (80+ hours/month) are in effect, as modeled for 2027. Parents with children under 13 are exempt from work requirements.

In [ ]:
from policyengine_us import Simulation
from policyengine_core.periods import instant
from policyengine_core.reforms import Reform

YEAR = 2027

def create_ut_medicaid_expansion_repeal():
    """Repeal Utah Medicaid expansion by setting income limit to -inf."""
    def modify_parameters(parameters):
        parameters.gov.hhs.medicaid.eligibility.categories.adult.income_limit.UT.update(
            start=instant(f"{YEAR}-01-01"),
            stop=instant("2100-12-31"),
            value=float("-inf"),
        )
        return parameters

    class reform(Reform):
        def apply(self):
            self.modify_parameters(modify_parameters)

    return reform

In [ ]:
def analyze_household(situation, name):
    """Analyze a household under baseline and reform."""
    baseline = Simulation(situation=situation)
    reformed = Simulation(situation=situation, reform=create_ut_medicaid_expansion_repeal())
    
    # Get results for each person
    people = list(situation["people"].keys())
    
    print(f"{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    
    for person in people:
        age = situation["people"][person].get("age", {}).get(YEAR, "N/A")
        
        # Baseline
        b_medicaid = baseline.calculate("medicaid", YEAR, map_to="person")[people.index(person)]
        b_ptc_elig = baseline.calculate("is_aca_ptc_eligible", YEAR)[people.index(person)]
        
        # Reform
        r_medicaid = reformed.calculate("medicaid", YEAR, map_to="person")[people.index(person)]
        r_ptc_elig = reformed.calculate("is_aca_ptc_eligible", YEAR)[people.index(person)]
        
        print(f"\n{person} (age {age}):")
        print(f"  Baseline:  Medicaid=${b_medicaid:,.0f}/yr, ACA eligible={b_ptc_elig}")
        print(f"  Reform:    Medicaid=${r_medicaid:,.0f}/yr, ACA eligible={r_ptc_elig}")
        
        if b_medicaid > 0 and r_medicaid == 0:
            if r_ptc_elig:
                print(f"  → LOSES MEDICAID, GAINS ACA ELIGIBILITY")
            else:
                print(f"  → FALLS INTO COVERAGE GAP (no coverage option)")
    
    # Tax unit level PTC
    b_ptc = baseline.calculate("premium_tax_credit", YEAR)[0]
    r_ptc = reformed.calculate("premium_tax_credit", YEAR)[0]
    
    print(f"\nHousehold Premium Tax Credit:")
    print(f"  Baseline: ${b_ptc:,.0f}/yr")
    print(f"  Reform:   ${r_ptc:,.0f}/yr")
    print()

## Household 1: Single Adult Below Poverty Line → Coverage Gap

A single adult earning $12,000/year (about 75% FPL). Under current law, they qualify for Medicaid expansion. If expansion is repealed, they fall into the coverage gap because ACA subsidies only start at 100% FPL.

In [ ]:
household_1 = {
    "people": {
        "adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: 12_000},
            "monthly_hours_worked": {YEAR: 100},  # Meets work requirement
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["adult"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["adult"],
        }
    },
    "households": {
        "household": {
            "members": ["adult"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["adult"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["adult"],
        }
    },
}

analyze_household(household_1, "HOUSEHOLD 1: Single adult, $12k/yr (75% FPL) → COVERAGE GAP")

## Household 2: Single Adult Above Poverty Line → ACA Transition

A single adult earning $18,000/year (about 112% FPL). Under current law, they qualify for Medicaid expansion. If expansion is repealed, they can transition to ACA marketplace coverage with premium subsidies.

In [ ]:
household_2 = {
    "people": {
        "adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: 18_000},
            "monthly_hours_worked": {YEAR: 100},  # Meets work requirement
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["adult"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["adult"],
        }
    },
    "households": {
        "household": {
            "members": ["adult"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["adult"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["adult"],
        }
    },
}

analyze_household(household_2, "HOUSEHOLD 2: Single adult, $18k/yr (112% FPL) → ACA TRANSITION")

## Household 3: Parent with Child Below Poverty Line → Coverage Gap for Parent

A single parent with one child, earning $15,000/year (about 68% FPL for family of 2). The parent loses Medicaid expansion and falls into the coverage gap. The child remains eligible for Medicaid/CHIP (children's eligibility is separate from expansion).

In [ ]:
household_3 = {
    "people": {
        "parent": {
            "age": {YEAR: 30},
            "employment_income": {YEAR: 15_000},
            "monthly_hours_worked": {YEAR: 100},  # Parent exempt (child under 13), but working anyway
        },
        "child": {
            "age": {YEAR: 8},
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent", "child"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["parent", "child"],
        }
    },
    "households": {
        "household": {
            "members": ["parent", "child"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["parent", "child"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["parent"],
        }
    },
}

analyze_household(household_3, "HOUSEHOLD 3: Single parent + child, $15k/yr (68% FPL) → PARENT IN COVERAGE GAP")

## Household 4: Couple with Children Above Poverty Line → ACA Transition

A married couple with two children, earning $38,000/year (about 115% FPL for family of 4). Both parents lose Medicaid expansion but can transition to ACA. Children remain on Medicaid/CHIP.

In [ ]:
household_4 = {
    "people": {
        "parent1": {
            "age": {YEAR: 40},
            "employment_income": {YEAR: 38_000},
            "monthly_hours_worked": {YEAR: 160},  # Full-time worker
        },
        "parent2": {
            "age": {YEAR: 38},
            "monthly_hours_worked": {YEAR: 0},  # Exempt (children under 13)
        },
        "child1": {
            "age": {YEAR: 12},
        },
        "child2": {
            "age": {YEAR: 7},
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent1", "parent2", "child1", "child2"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["parent1", "parent2", "child1", "child2"],
        }
    },
    "households": {
        "household": {
            "members": ["parent1", "parent2", "child1", "child2"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["parent1", "parent2", "child1", "child2"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["parent1", "parent2"],
        }
    },
}

analyze_household(household_4, "HOUSEHOLD 4: Married couple + 2 kids, $38k/yr (115% FPL) → ACA TRANSITION")

## Summary

| Household | Income | % FPL | Outcome |
|-----------|--------|-------|--------|
| Single adult | $12,000 | 75% | **Coverage gap** - no options |
| Single adult | $18,000 | 112% | ACA transition - gets subsidies |
| Parent + child | $15,000 | 68% | **Parent in coverage gap**, child keeps Medicaid |
| Couple + 2 kids | $38,000 | 115% | Parents get ACA, kids keep Medicaid |

**The 100% FPL threshold is critical:** Below it, there's no coverage option. Above it, ACA subsidies are available.